In [143]:
# 라이브러리 임포트
import os
import pandas as pd
import dask.dataframe as dd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

In [145]:
filepath = 'G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\'

In [147]:
df_1 = er.Read_Chunk('G:\\내 드라이브\\DataSet\\서울특별시 공공자전거 대여이력 정보\\2020\\공공자전거 대여이력 정보_2020.01.csv')

 Processed a chunk of size: 765300


In [149]:
df_1.head()

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-04061,2020-01-01 00:01:04,429,송도병원,2,2020-01-01 00:04:21,00372,약수역 3번출구 뒤,8,2.0,0.0
1,SPB-06686,2020-01-01 00:02:53,1637,KT 노원점 건물 앞,14,2020-01-01 00:04:48,01656,중앙하이츠 아파트 입구,9,1.0,350.0
2,SPB-15937,2020-01-01 00:01:21,1924,삼부르네상스파크빌,10,2020-01-01 00:05:51,01955,디지털입구 교차로,7,4.0,800.0
3,SPB-14805,2020-01-01 00:03:12,437,대흥역 1번출구,1,2020-01-01 00:05:54,00126,서강대 후문 옆,18,2.0,0.0
4,SPB-09038,2020-01-01 00:01:11,1168,마곡엠밸리10단지 앞,5,2020-01-01 00:05:57,01152,마곡역교차로,2,4.0,660.0


In [151]:
df_1.columns

Index(['자전거번호', '대여일시', '대여 대여소번호', '대여 대여소명', '대여거치대', '반납일시', '반납대여소번호',
       '반납대여소명', '반납거치대', '이용시간', '이용거리'],
      dtype='object')

In [155]:
start_time = time.time()
_columns = ['자전거번호', '대여일자', '대여시간', '반납일자', '반납시간', '이용시간(분)', '이용거리(M)']
df_parquet_20_24 = dd.read_parquet(filepath + '서울특별시 공공자전거 대여이력 정보_20~24.parquet', columns=_columns)
print("read_parquet time :", time.time() - start_time)

read_parquet time : 0.025887489318847656


In [157]:
from dask.diagnostics import ProgressBar
from time import sleep
pbar = ProgressBar()
pbar.register()

start_time = time.time()
# df_parquet_20_24 = df_parquet_20_24[df_parquet_20_24['대여일자'].between('2020-01-01', '2020-12-31')].compute()
df_parquet_20_24 = df_parquet_20_24.compute()
print("compute time :", time.time() - start_time)

[########################################] | 100% Completed | 260.94 s
[########################################] | 100% Completed | 261.01 s
[########################################] | 100% Completed | 261.01 s
[########################################] | 100% Completed | 261.03 s
[########################################] | 100% Completed | 261.03 s
[########################################] | 100% Completed | 261.11 s
[########################################] | 100% Completed | 261.13 s
[########################################] | 100% Completed | 261.20 s
[########################################] | 100% Completed | 261.22 s
[########################################] | 100% Completed | 261.30 s
compute time : 261.43506383895874


In [159]:
df_parquet_20_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164164195 entries, 0 to 164164194
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   자전거번호    string 
 1   대여일자     string 
 2   대여시간     object 
 3   반납일자     string 
 4   반납시간     object 
 5   이용시간(분)  float64
 6   이용거리(M)  float64
dtypes: float64(2), object(2), string(3)
memory usage: 13.0+ GB


In [11]:
df_parquet_20_24.count()

자전거번호      164164195
대여일자       164164195
대여시간       164164195
반납일자       164164195
반납시간       164164195
이용시간(분)    164164195
이용거리(M)    164163969
dtype: int64

In [13]:
df_parquet_20_24.head()

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M)
0,SPB-04061,2020-01-01,00:01:04,2020-01-01,00:04:21,2.0,0.0
1,SPB-06686,2020-01-01,00:02:53,2020-01-01,00:04:48,1.0,350.0
2,SPB-15937,2020-01-01,00:01:21,2020-01-01,00:05:51,4.0,800.0
3,SPB-14805,2020-01-01,00:03:12,2020-01-01,00:05:54,2.0,0.0
4,SPB-09038,2020-01-01,00:01:11,2020-01-01,00:05:57,4.0,660.0


In [17]:
df_parquet_20_24['년'] = df_parquet_20_24['대여일자'].str.slice(start = 0, stop = 4)

### 이용거리(M) == 0 인 데이터 확인

In [15]:
df_parquet_20_24[df_parquet_20_24['이용거리(M)'] == 0]

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M)
0,SPB-04061,2020-01-01,00:01:04,2020-01-01,00:04:21,2.0,0.0
3,SPB-14805,2020-01-01,00:03:12,2020-01-01,00:05:54,2.0,0.0
18,SPB-17241,2020-01-01,00:06:05,2020-01-01,00:10:06,3.0,0.0
43,SPB-25160,2020-01-01,00:08:17,2020-01-01,00:15:51,6.0,0.0
53,SPB-13778,2020-01-01,00:01:33,2020-01-01,00:17:52,15.0,0.0
...,...,...,...,...,...,...,...
164163696,SPB-51425,2024-06-30,23:59:18,2024-07-01,00:21:45,22.0,0.0
164163801,SPB-50953,2024-06-30,22:18:27,2024-07-01,00:27:12,129.0,0.0
164163927,SPB-54387,2024-06-30,23:42:52,2024-07-01,00:40:12,57.0,0.0
164164188,SPB-82207,2024-06-30,23:15:58,2024-07-01,23:23:58,45.0,0.0


In [19]:
df_parquet_20_24.head()

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M),년
0,SPB-04061,2020-01-01,00:01:04,2020-01-01,00:04:21,2.0,0.0,2020
1,SPB-06686,2020-01-01,00:02:53,2020-01-01,00:04:48,1.0,350.0,2020
2,SPB-15937,2020-01-01,00:01:21,2020-01-01,00:05:51,4.0,800.0,2020
3,SPB-14805,2020-01-01,00:03:12,2020-01-01,00:05:54,2.0,0.0,2020
4,SPB-09038,2020-01-01,00:01:11,2020-01-01,00:05:57,4.0,660.0,2020


In [ ]:
df_parquet_20_24[df_parquet_20_24['이용거리(M)'] == 0].groupby('년').sum().reset_index()

---
### 년도별 (df_parquet['이용거리(M)'] == 0) 데이터 각각확인
 - 2020 : 15420304 / 23287911 : 66.21591777811243
 - 2021 : 3733980 / 30788788  : 12.127726495762028
 - 2022  2286444 / 41591732  : 5.497352214137175:
 - 2023  2186651 / 46388223  : 4.713806346925598:
 - 2024  1111191 / 22107541  : 5.026298492446537:

In [102]:
chk_year = '2024'
start_time = time.time()
_columns = ['자전거번호', '대여일자', '대여시간', '반납일자', '반납시간', '이용시간(분)', '이용거리(M)']
df_parquet = dd.read_parquet(filepath + '서울특별시 공공자전거 대여이력 정보_' + chk_year + '.parquet', columns=_columns)
print("read_parquet time :", time.time() - start_time)

read_parquet time : 14.823721885681152


In [104]:
from dask.diagnostics import ProgressBar
from time import sleep
pbar = ProgressBar()
pbar.register()

start_time = time.time()
df_parquet = df_parquet.compute()
print("compute time :", time.time() - start_time)

[########################################] | 100% Completed | 18.76 s
[########################################] | 100% Completed | 18.85 s
[########################################] | 100% Completed | 18.95 s
[########################################] | 100% Completed | 19.05 s
[########################################] | 100% Completed | 19.15 s
[########################################] | 100% Completed | 19.15 s
[########################################] | 100% Completed | 19.25 s
[########################################] | 100% Completed | 19.35 s
compute time : 19.387656927108765


In [106]:
df_parquet.head()

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M)
0,SPB-46712,2024-01-01,00:04:10,2024-01-01,00:05:13,1,0.00
1,SPB-35089,2024-01-01,00:00:10,2024-01-01,00:06:23,6,1147.58
2,SPB-36386,2024-01-01,00:03:13,2024-01-01,00:06:33,3,642.13
3,SPB-61278,2024-01-01,00:02:42,2024-01-01,00:07:24,4,830.00
4,SPB-36848,2024-01-01,00:03:02,2024-01-01,00:07:33,4,791.08


In [108]:
df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22107541 entries, 0 to 22107540
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   자전거번호    string 
 1   대여일자     string 
 2   대여시간     object 
 3   반납일자     string 
 4   반납시간     object 
 5   이용시간(분)  int64  
 6   이용거리(M)  float64
dtypes: float64(1), int64(1), object(2), string(3)
memory usage: 1.8+ GB


In [110]:
df_parquet.count()

자전거번호      22107541
대여일자       22107541
대여시간       22107541
반납일자       22107541
반납시간       22107541
이용시간(분)    22107541
이용거리(M)    22107541
dtype: int64

In [112]:
df_parquet.isnull().sum()

자전거번호      0
대여일자       0
대여시간       0
반납일자       0
반납시간       0
이용시간(분)    0
이용거리(M)    0
dtype: int64

In [114]:
df_parquet_0 = df_parquet

In [116]:
print(df_parquet_0.shape[0])
print(df_parquet_0[df_parquet_0['이용거리(M)'] == 0].shape[0])
print(df_parquet_0[df_parquet_0['이용거리(M)'] == 0].shape[0] / (df_parquet_0.shape[0] / 100))

22107541
1111191
5.026298492446537


In [121]:
df_parquet_0['이용속도'] = df_parquet_0['이용거리(M)'] / df_parquet_0['이용시간(분)']

In [124]:
df_parquet_0.head()

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M),이용속도
0,SPB-46712,2024-01-01,00:04:10,2024-01-01,00:05:13,1,0.00,0.000000
1,SPB-35089,2024-01-01,00:00:10,2024-01-01,00:06:23,6,1147.58,191.263333
2,SPB-36386,2024-01-01,00:03:13,2024-01-01,00:06:33,3,642.13,214.043333
3,SPB-61278,2024-01-01,00:02:42,2024-01-01,00:07:24,4,830.00,207.500000
4,SPB-36848,2024-01-01,00:03:02,2024-01-01,00:07:33,4,791.08,197.770000


In [134]:
cond1 = df_parquet_0['이용시간(분)'] == 5
cond2 = df_parquet_0['이용거리(M)'] < 100
df_parquet_0[cond1 & cond2]

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M),이용속도
472,SPB-82851,2024-01-01,00:34:25,2024-01-01,00:39:03,5,0.00,0.000
1392,SPB-60444,2024-01-01,01:12:54,2024-01-01,01:18:00,5,30.36,6.072
2788,SPB-36772,2024-01-01,01:58:57,2024-01-01,02:04:24,5,42.64,8.528
3382,SPB-52065,2024-01-01,02:17:28,2024-01-01,02:23:03,5,60.54,12.108
3722,SPB-54032,2024-01-01,02:30:00,2024-01-01,02:35:53,5,51.57,10.314
...,...,...,...,...,...,...,...,...
22098383,SPB-39800,2024-06-30,22:28:20,2024-06-30,22:33:30,5,73.34,14.668
22098777,SPB-34684,2024-06-30,22:30:49,2024-06-30,22:36:43,5,0.10,0.020
22103325,SPB-80159,2024-06-30,23:16:56,2024-06-30,23:22:06,5,90.00,18.000
22105481,SPB-39772,2024-06-30,23:44:08,2024-06-30,23:50:00,5,32.34,6.468


In [136]:
cond1 = df_parquet_0['이용시간(분)'] == 5
df_parquet_0[cond1].shape[0]

1350132

In [140]:
cond1 = df_parquet_0['이용거리(M)'] == 0
df_parquet_0[cond1]

,자전거번호,대여일자,대여시간,반납일자,반납시간,이용시간(분),이용거리(M),이용속도
0,SPB-46712,2024-01-01,00:04:10,2024-01-01,00:05:13,1,0.0,0.0
9,SPB-63233,2024-01-01,00:07:59,2024-01-01,00:08:16,0,0.0,NaN
10,SPB-51653,2024-01-01,00:07:49,2024-01-01,00:08:25,0,0.0,NaN
18,SPB-64748,2024-01-01,00:08:39,2024-01-01,00:08:48,0,0.0,NaN
24,SPB-52471,2024-01-01,00:07:04,2024-01-01,00:09:37,2,0.0,0.0
...,...,...,...,...,...,...,...,...
22107042,SPB-51425,2024-06-30,23:59:18,2024-07-01,00:21:45,22,0.0,0.0
22107147,SPB-50953,2024-06-30,22:18:27,2024-07-01,00:27:12,129,0.0,0.0
22107273,SPB-54387,2024-06-30,23:42:52,2024-07-01,00:40:12,57,0.0,0.0
22107534,SPB-82207,2024-06-30,23:15:58,2024-07-01,23:23:58,45,0.0,0.0
